## Setup
#### Load the API key and relevant Python libaries.

##### this notebook runs in Colab

In [1]:
from collections import defaultdict
import json
import numpy as np
import pandas as pd
import os

import io
from dotenv import dotenv_values
import openai
import os
from copy import deepcopy
import time
import ast
import re
import pprint
from openai import OpenAI

# LLM Open AI

In [2]:
# Get the first key from the uploaded dictionary
env_file_key = "../../auixiliary/env_GENERAL"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

# Load the content into a variable
env_variables = dotenv_values(stream=io.StringIO(env_content))

api_key = env_variables['OPENAI_API_KEY']
# openai.api_key = api_key

client = OpenAI(
    # This is the default and can be omitted
    api_key=api_key,
)


# Models

In [3]:
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4-turbo-2024-04-09",
        messages=prompt
    )
    return response.choices[0].message.content.strip()

# Iterate and Save Use Riskiness Results

# Functions

In [4]:
def replace_key(d, old_key, new_key):
  """
  Replace `old_key` with `new_key` in dictionary `d`.
  The associated value is retained.
  """
  if old_key in d:
      d[new_key] = d.pop(old_key)
  return d

## Read In Prompt Result

In [5]:
def read_prompt_output(input_data_location):
  
  # Open the JSON file for reading
  with open(input_data_location, 'r') as file:
      # Load JSON data from file
      file_content = json.load(file)

  return file_content

## READ DATA

In [32]:
uses_of = "HDI"

In [33]:
data_location = "../../data/"
USES = read_prompt_output(data_location + f"LLM_uses_{uses_of}.json")

In [34]:
USES[1]

{'Use': 2,
 'Domain': 'Biometric identification and categorization of natural persons',
 'Purpose': 'Crowd monitoring and control',
 'Capability': 'Identifying and tracking individuals in crowds',
 'AI User': 'Law enforcement agencies',
 'AI Subject': 'Individuals in public spaces'}

# FINAL PROMPT

In [35]:
# Assuming you have the variables domain, purpose, aiCapability, aiUser, and aiSubject defined with appropriate values

MESSAGES = [
    {
        'role': 'system',
        'content': """You are an experienced regulatory compliance specialists who works in the field of AI technology regulation. You are thoughtful, decisive, experienced and conscientious.
        You have access to the entirety of the EU AI Act and its amendments, which outline how various AI technologies are to be regulated and risk-classified within the European Union."""
    },
    {
        'role': 'user',
        'content': """

        Classify the following AI system by utilizing a three-tier classification: 1) Prohibited, 2) High Risk, and 3) Limited or Low Risk.

        Follow these four steps below:
        1. Write a brief description of the AI system, using similar language to the EU AI Act. The description should start with "The AI system intended to be used ...", and be written in a single sentence.
        2. Determine whether the AI system is Prohibited or of High Risk, providing the exact text from the EU AI Act and explaining the reasoning. Be very strict and verify the reasoning.
        Assume High Risk unless there is clear evidence that it is Prohibited. Pay particular attention to the subject and user of the AI system, as this is critical for classification.
        Ensure that the subject and user align with the text. They are very important. Also, ensure that you understand the purpose and the capability of the AI system as this is highly critical for the risk classification.
        For example, the capability to verify patient identities by using AI technology implies the use of biometric identification of patients. Be aware of these and similar cases.
        3. Go through all the amendments to the EU AI Act and ensure that nothing has changed that would affect the classification.
        If something has changed, update the classification accordingly and explicitly reference the amendment that most closely resembles the AI system.
        The amendments can be found under the text: "Here are some important amendments to the Act:"
        4. If the AI system is neither Prohibited nor High Risk, classify it as Limited or Low Risk.

        It is of utmost importance to exercise precision and make accurate judgments when classifying the risk associated with the AI system.
        Please carefully consider all the regulations listed below during the risk classification of the AI system:

    The relevant portions of the Act for what is Prohibited:
  
      Article 5: Prohibited Artificial Intelligence Practices

      1. The following artificial intelligence practices shall be prohibited:
        (a) the placing on the market, putting into service or use of an AI system that deploys subliminal techniques beyond a person’s consciousness or purposefully manipulative or deceptive techniques, with the objective to or the effect of materially distorting a person’s or a group of persons’ behaviour by appreciably impairing the person’s ability to make an informed decision, thereby causing the person to take a decision that that person would not have otherwise taken in a manner that causes or is likely to cause that person, another person or group of persons significant harm;
        (b) the placing on the market, putting into service or use of an AI system that exploits any of the vulnerabilities of a person or a specific group of persons due to their age, disability or a specific social or economic situation, with the objective to or the effect of materially distorting the behaviour of that person or a person pertaining to that group in a manner that causes or is reasonably likely to cause that person or another person significant harm;
        (ba) the placing on the market or putting into service for this specific purpose, or use of biometric categorisation systems that categorise individually natural persons based on their biometric data to deduce or infer their race, political opinions, trade union membership, religious or philosophical beliefs, sex life or sexual orientation. This prohibition does not cover any labelling or filtering of lawfully acquired biometric datasets, such as images, based on biometric data or categorizing of biometric data in the area of law enforcement;
        (c) the placing on the market, putting into service or use of AI systems for the evaluation or classification of natural persons or groups thereof over a certain period of time based on their social behaviour or known, inferred or predicted personal or personality characteristics, with the social score leading to either or both of the following:
          (i) detrimental or unfavourable treatment of certain natural persons or whole groups thereof in social contexts that are unrelated to the contexts in which the data was originally generated or collected;
          (ii) detrimental or unfavourable treatment of certain natural persons or groups thereof that is unjustified or disproportionate to their social behaviour or its gravity;
        (d) the use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for the purpose of law enforcement unless and in as far as such use is strictly necessary for one of the following objectives:
          (i) the targeted search for specific victims of abduction, trafficking in human beings and sexual exploitation of human beings as well as search for missing persons;
          (ii) the prevention of a specific, substantial and imminent threat to the life or physical safety of natural persons or a genuine and present or genuine and foreseeable threat of a terrorist attack;
          (iii) the localisation or identification of a person suspected of having committed a criminal offence, for the purposes of conducting a criminal investigation, prosecution or executing a criminal penalty for offences, referred to in Annex II and punishable in the Member State concerned by a custodial sentence or a detention order for a maximum period of at least four years. This paragraph is without prejudice to the provisions in Article 9 of the GDPR for the processing of biometric data for purposes other than law enforcement;
        (da) the placing on the market, putting into service for this specific purpose, or use of an AI system for making risk assessments of natural persons in order to assess or predict the risk of a natural person to commit a criminal offence, based solely on the profiling of a natural person or on assessing their personality traits and characteristics; This prohibition shall not apply to AI systems used to support the human assessment of the involvement of a person in a criminal activity, which is already based on objective and verifiable facts directly linked to a criminal activity;
        (db) the placing on the market, putting into service for this specific purpose, or use of AI systems that create or expand facial recognition databases through the untargeted scraping of facial images from the internet or CCTV footage;
        (dc) the placing on the market, putting into service for this specific purpose, or use of AI systems to infer emotions of a natural person in the areas of workplace and education institutions except in cases where the use of the AI system is intended to be put in place or into the market for medical or safety reasons;

      1a. This Article shall not affect the prohibitions that apply where an artificial intelligence practice infringes other Union law.

      2. The use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for the purpose of law enforcement for any of the objectives referred to in paragraph 1 point d) shall only be deployed for the purposes under paragraph 1, point d) to confirm the specifically targeted individual’s identity and it shall take into account the following elements:
        (a) the nature of the situation giving rise to the possible use, in particular the seriousness, probability and scale of the harm caused in the absence of the use of the system;
        (b) the consequences of the use of the system for the rights and freedoms of all persons concerned, in particular the seriousness, probability and scale of those consequences. In addition, the use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for the purpose of law enforcement for any of the objectives referred to in paragraph 1 point d) shall comply with necessary and proportionate safeguards and conditions in relation to the use in accordance with national legislations authorizing the use thereof, in particular as regards the temporal, geographic and personal limitations. The use of the ‘real-time’ remote biometric identification system in publicly accessible spaces shall only be authorised if the law enforcement authority has completed a fundamental rights impact assessment as provided for in Article 27 and has registered the system in the database according to Article 49. However, in duly justified cases of urgency, the use of the system may be commenced without the registration, provided that the registration is completed without undue delay.
      3. As regards paragraphs 1, point (d) and 2, each use for the purpose of law enforcement of a ‘real-time’ remote biometric identification system in publicly accessible spaces shall be subject to a prior authorisation granted by a judicial authority or an independent administrative authority whose decision is binding of the Member State in which the use is to take place, issued upon a reasoned request and in accordance with the detailed rules of national law referred to in paragraph 4. However, in a duly justified situation of urgency, the use of the system may be commenced without an authorisation provided that, such authorisation shall be requested without undue delay, at the latest within 24 hours. If such authorisation is rejected, its use shall be stopped with immediate effect and all the data, as well as the results and outputs of this use shall be immediately discarded and deleted. The competent judicial authority or an independent administrative authority whose decision is binding shall only grant the authorisation where it is satisfied, based on objective evidence or clear indications presented to it, that the use of the ‘real-time’ remote biometric identification system at issue is necessary for and proportionate to achieving one of the objectives specified in paragraph 1, point (d), as identified in the request and, in particular, remains limited to what is strictly necessary concerning the period of time as well as geographic and personal scope. In deciding on the request, the competent judicial authority or an independent administrative authority whose decision is binding shall take into account the elements referred to in paragraph 2. It shall be ensured that no decision that produces an adverse legal effect on a person may be taken by the judicial authority or an independent administrative authority whose decision is binding solely based on the output of the remote biometric identification system.
      3a. Without prejudice to paragraph 3, each use of a ‘real-time’ remote biometric identification system in publicly accessible spaces for law enforcement purposes shall be notified to the relevant market surveillance authority and the national data protection authority in accordance with the national rules referred to in paragraph 4. The notification shall as a minimum contain the information specified under paragraph 5 and shall not include sensitive operational data.
      4. A Member State may decide to provide for the possibility to fully or partially authorise the use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for the purpose of law enforcement within the limits and under the conditions listed in paragraphs 1, point (d), 2 and 3. Member States concerned shall lay down in their national law the necessary detailed rules for the request, issuance and exercise of, as well as supervision and reporting relating to, the authorisations referred to in paragraph 3. Those rules shall also specify in respect of which of the objectives listed in paragraph 1, point (d), including which of the criminal offences referred to in point (iii) thereof, the competent authorities may be authorised to use those systems for the purpose of law enforcement. Member States shall notify those rules to the Commission at the latest 30 days following the adoption thereof. Member States may introduce, in accordance with Union law, more restrictive laws on the use of remote biometric identification systems.
      5. National market surveillance authorities and the national data protection authorities of Member States that have been notified of the use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for law enforcement purposes pursuant to paragraph 3a shall submit to the Commission annual reports on such use. For that purpose, the Commission shall provide Member States and national market surveillance and data protection authorities with a template, including information on the number of the decisions taken by competent judicial authorities or an independent administrative authority whose decision is binding upon requests for authorisations in accordance with paragraph 3 and their result.
      6. The Commission shall publish annual reports on the use of ‘real-time’ remote biometric identification systems in publicly accessible spaces for law enforcement purposes based on aggregated data in Member States based on the annual reports referred to in paragraph 5, which shall not include sensitive operational data of the related law enforcement activities.

    (*) For the purposes of this Regulation the notion of *publicly accessible* space should be understood as referring to any physical place that is accessible to an undetermined number of natural persons, and irrespective of whether the place in question is privately or publicly owned and irrespective of the activity for which the place may be used, such as commerce (for instance, shops, restaurants, cafés), services (for instance, banks, professional activities, hospitality), sport (for instance, swimming pools, gyms, stadiums), transport (for instance, bus, metro and railway stations, airports, means of transport ), entertainment (for instance, cinemas, theatres, museums, concert and conference halls) leisure or otherwise (for instance, public roads and squares, parks, forests, playgrounds). A place should be classified as publicly accessible also if, regardless of potential capacity or security restrictions, access is subject to certain predetermined conditions, which can be fulfilled by an undetermined number of persons, such as purchase of a ticket or title of transport, prior registration or having a certain age. By contrast, a place should not be considered publicly accessible if access is limited to specific and defined natural persons through either Union or national law directly related to public safety or security or through the clear manifestation of will by the person having the relevant authority on the place. The factual possibility of access alone (e.g. an unlocked door, an open gate in a fence) does not imply that the place is publicly accessible in the presence of indications or circumstances suggesting the contrary (e.g. signs prohibiting or restricting access). Company and factory premises as well as offices and workplaces that are intended to be accessed only by relevant employees and service providers are places that are not publicly accessible. Publicly accessible spaces should not include prisons or border control. Some other areas may be composed of both not publicly accessible and publicly accessible areas, such as the hallway of a private residential building necessary to access a doctor's office or an airport. Online spaces are not covered either, as they are not physical spaces. Whether a given space is accessible to the public should however be determined on a case-by-case basis, having regard to the specificities of the individual situation at hand.

    
  The relevant portions of the Act for what is High Risk:

    Article 6: Classification Rules for High-Risk AI Systems
      1. Irrespective of whether an AI system is placed on the market or put into service independently from the products referred to in points (a) and (b), that AI system shall be considered high-risk where both of the following conditions are fulfilled:
        (a) the AI system is intended to be used as a safety component of a product, or the AI system is itself a product, covered by the Union harmonisation legislation listed in Annex I;
        (b) the product whose safety component pursuant to point (a) is the AI system, or the AI system itself as a product, is required to undergo a third party conformity assessment, with a view to the placing on the market or putting into service of that product pursuant to the Union harmonisation legislation listed in Annex I.
      2. In addition to the high-risk AI systems referred to in paragraph 1, AI systems referred to in Annex III shall also be considered high-risk.
      2a. By derogation from paragraph 2 AI systems shall not be considered as high risk if they do not pose a significant risk of harm, to the health, safety or fundamental rights of natural persons, including by not materially influencing the outcome of decision making. This shall be the case if one or more of the following criteria are fulfilled:
        (a) the AI system is intended to perform a narrow procedural task;
        (b) the AI system is intended to improve the result of a previously completed human activity;
        (c) the AI system is intended to detect decision-making patterns or deviations from prior decision-making patterns and is not meant to replace or influence the previously completed human assessment, without proper human review; or
        (d) the AI system is intended to perform a preparatory task to an assessment relevant for the purpose of the use cases listed in Annex III. Notwithstanding first subparagraph of this paragraph, an AI system shall always be considered high-risk if the AI system performs profiling of natural persons.
      2b. A provider who considers that an AI system referred to in Annex III is not high-risk shall document its assessment before that system is placed on the market or put into service. Such provider shall be subject to the registration obligation set out in Article 49(1a). Upon request of national competent authorities, the provider shall provide the documentation of the assessment.
      2c. The Commission shall, after consulting the AI Board, and no later than [18 months] after the entry into force of this Regulation, provide guidelines specifying the practical implementation of this article completed by a comprehensive list of practical examples of high risk and non-high risk use cases on AI systems pursuant to Article 82b.
      2d. The Commission is empowered to adopt delegated acts in accordance with Article 97 to amend the criteria laid down in points a) to d) of the first subparagraph of paragraph 2a. The Commission may adopt delegated acts adding new criteria to those laid down in points a) to d) of the first subparagraph of paragraph 2a, or modifying them, only where there is concrete and reliable evidence of the existence of AI systems that fall under the scope of Annex III but that do not pose a significant risk of harm to the health, safety and fundamental rights. The Commission shall adopt delegated acts deleting any of the criteria laid down in the first subparagraph of paragraph 2a where there is concrete and reliable evidence that this is necessary for the purpose of maintaining the level of protection of health, safety and fundamental rights in the Union. Any amendment to the criteria laid down in points a) to d) set out in the first subparagraph of paragraph 2a shall not decrease the overall level of protection of health, safety and fundamental rights in the Union. When adopting the delegated acts, the Commission shall ensure consistency with the delegated acts adopted pursuant to Article 7(1) and shall take account of market and technological developments.


    Annex I: List of Union Harmonisation Legislation

    Part I Section A. List of Union harmonisation legislation based on the New Legislative Framework
    1. Directive 2006/42/EC of the European Parliament and of the Council of 17 May 2006 on machinery, and amending Directive 95/16/EC (OJ L 157, 9.6.2006, p. 24) [as repealed by the Machinery Regulation].
    2. Directive 2009/48/EC of the European Parliament and of the Council of 18 June 2009 on the safety of toys (OJ L 170, 30.6.2009, p. 1).
    3. Directive 2013/53/EU of the European Parliament and of the Council of 20 November 2013 on recreational craft and personal watercraft and repealing Directive 94/25/EC (OJ L 354, 28.12.2013, p. 90).
    4. Directive 2014/33/EU of the European Parliament and of the Council of 26 February 2014 on the harmonisation of the laws of the Member States relating to lifts and safety components for lifts (OJ L 96, 29.3.2014, p. 251).
    5. Directive 2014/34/EU of the European Parliament and of the Council of 26 February 2014 on the harmonisation of the laws of the Member States relating to equipment and protective systems intended for use in potentially explosive atmospheres (OJ L 96, 29.3.2014, p. 309).
    6. Directive 2014/53/EU of the European Parliament and of the Council of 16 April 2014 on the harmonisation of the laws of the Member States relating to the making available on the market of radio equipment and repealing Directive 1999/5/EC (OJ L 153, 22.5.2014, p. 62).
    7. Directive 2014/68/EU of the European Parliament and of the Council of 15 May 2014 on the harmonisation of the laws of the Member States relating to the making available on the market of pressure equipment (OJ L 189, 27.6.2014, p. 164).
    8. Regulation (EU) 2016/424 of the European Parliament and of the Council of 9 March 2016 on cableway installations and repealing Directive 2000/9/EC (OJ L 81, 31.3.2016, p. 1).
    9. Regulation (EU) 2016/425 of the European Parliament and of the Council of 9 March 2016 on personal protective equipment and repealing Council Directive 89/686/EEC (OJ L 81, 31.3.2016, p. 51).
    10. Regulation (EU) 2016/426 of the European Parliament and of the Council of 9 March 2016 on appliances burning gaseous fuels and repealing Directive 2009/142/EC (OJ L 81, 31.3.2016, p. 99).
    11. Regulation (EU) 2017/745 of the European Parliament and of the Council of 5 April 2017 on medical devices, amending Directive 2001/83/EC, Regulation (EC) No 178/2002 and Regulation (EC) No 1223/2009 and repealing Council Directives 90/385/EEC and 93/42/EEC (OJ L 117, 5.5.2017, p. 1.
    12. Regulation (EU) 2017/746 of the European Parliament and of the Council of 5 April 2017 on in vitro diagnostic medical devices and repealing Directive 98/79/EC and Commission Decision 2010/227/EU (OJ L 117, 5.5.2017, p. 176).

    Part II Section B. List of other Union harmonisation legislation
    13. Regulation (EC) No 300/2008 of the European Parliament and of the Council of 11 March 2008 on common rules in the field of civil aviation security and repealing Regulation (EC) No 2320/2002 (OJ L 97, 9.4.2008, p. 72).
    14. Regulation (EU) No 168/2013 of the European Parliament and of the Council of 15 January 2013 on the approval and market surveillance of two- or three-wheel vehicles and quadricycles (OJ L 60, 2.3.2013, p. 52).
    15. Regulation (EU) No 167/2013 of the European Parliament and of the Council of 5 February 2013 on the approval and market surveillance of agricultural and forestry vehicles (OJ L 60, 2.3.2013, p. 1).
    16. Directive 2014/90/EU of the European Parliament and of the Council of 23 July 2014 on marine equipment and repealing Council Directive 96/98/EC (OJ L 257, 28.8.2014, p. 146).
    17. Directive (EU) 2016/797 of the European Parliament and of the Council of 11 May 2016 on the interoperability of the rail system within the European Union (OJ L 138, 26.5.2016, p. 44).
    18. Regulation (EU) 2018/858 of the European Parliament and of the Council of 30 May 2018 on the approval and market surveillance of motor vehicles and their trailers, and of systems, components and separate technical units intended for such vehicles, amending Regulations (EC) No 715/2007 and (EC) No 595/2009 and repealing Directive 2007/46/EC (OJ L 151, 14.6.2018, p. 1).
    18a. Regulation (EU) 2019/2144 of the European Parliament and of the Council of 27 November 2019 on type-approval requirements for motor vehicles and their trailers, and systems, components and separate technical units intended for such vehicles, as regards their general safety and the protection of vehicle occupants and vulnerable road users, amending Regulation (EU) 2018/858 of the European Parliament and of the Council and repealing Regulations (EC) No 78/2009, (EC) No 79/2009 and (EC) No 661/2009 of the European Parliament and of the Council and Commission Regulations (EC) No 631/2009, (EU) No 406/2010, (EU) No 672/2010, (EU) No 1003/2010, (EU) No 1005/2010, (EU) No 1008/2010, (EU) No 1009/2010, (EU) No 19/2011, (EU) No 109/2011, (EU) No 458/2011, (EU) No 65/2012, (EU) No 130/2012, (EU) No 347/2012, (EU) No 351/2012, (EU) No 1230/2012 and (EU) 2015/166 (OJ L 325, 16.12.2019, p. 1).
    19. Regulation (EU) 2018/1139 of the European Parliament and of the Council of 4 July 2018 on common rules in the field of civil aviation and establishing a European Union Aviation Safety Agency, and amending Regulations (EC) No 2111/2005, (EC) No 1008/2008, (EU) No 996/2010, (EU) No 376/2014 and Directives 2014/30/EU and 2014/53/EU of the European Parliament and of the Council, and repealing Regulations (EC) No 552/2004 and (EC) No 216/2008 of the European Parliament and of the Council and Council Regulation (EEC) No 3922/91 (OJ L 212, 22.8.2018, p. 1), in so far as the design, production and placing on the market of aircrafts referred to in points (a) and (b) of Article 2(1) thereof, where it concerns unmanned aircraft and their engines, propellers, parts and equipment to control them remotely, are concerned.

    Annex II: List of Criminal Offences
    Original title: Annex II: List of Criminal Offences Referred to in Article 5(1)(iii)
      Terrorism,
      Trafficking in human beings,
      Sexual exploitation of children and child pornography,
      Illicit trafficking in narcotic drugs and psychotropic substances,
      Illicit trafficking in weapons,
      Munitions and explosives,
      Murder,
      Grievous bodily injury,
      Illicit trade in human organs and tissue,
      Illicit trafficking in nuclear or radioactive materials,
      Kidnapping, illegal restraint and hostage-taking,
      Crimes within the jurisdiction of the International Criminal Court,
      Unlawful seizure of aircraft/ships,
      Rape,
      Environmental crime,
      Organised or armed robbery,
      Sabotage,
      Participation in a criminal organisation involved in one or more offences listed above.

    Annex III: High-Risk AI Systems Referred to in Article 6(2) 
    High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas:
      1. Biometrics, insofar as their use is permitted under relevant Union or national law:
        (a) Remote biometric identification systems. This shall not include AI systems intended to be used for biometric verification whose sole purpose is to confirm that a specific natural person is the person he or she claims to be;
        (aa) AI systems intended to be used for biometric categorisation, according to sensitive or protected attributes or characteristics based on the inference of those attributes or characteristics;
        (ab) AI systems intended to be used for emotion recognition.
      2. Critical infrastructure:
        (a) AI systems intended to be used as safety components in the management and operation of critical digital infrastructure, road traffic and the supply of water, gas, heating and electricity.
      3. Education and vocational training:
        (a) AI systems intended to be used to determine access or admission or to assign natural persons to educational and vocational training institutions at all levels;
        (b) AI systems intended to be used to evaluate learning outcomes, including when those outcomes are used to steer the learning process of natural persons in educational and vocational training institutions at all levels;
        (ba) AI systems intended to be used for the purpose of assessing the appropriate level of education that individual will receive or will be able to access, in the context of/within education and vocational training institution;
        (bb) AI systems intended to be used for monitoring and detecting prohibited behaviour of students during tests in the context of/within education and vocational training institutions.
      4. Employment, workers management and access to self-employment:
        (a) AI systems intended to be used for recruitment or selection of natural persons, notably to place targeted job advertisements, to analyse and filter job applications, and to evaluate candidates;
        (b) AI intended to be used to make decisions affecting terms of the work related relationships, promotion and termination of work-related contractual relationships, to allocate tasks based on individual behaviour or personal traits or characteristics and to monitor and evaluate performance and behaviour of persons in such relationships.
      5. Access to and enjoyment of essential private services and essential public services and benefits:
        (a) AI systems intended to be used by public authorities or on behalf of public authorities to evaluate the eligibility of natural persons for essential public assistance benefits and services, including healthcare services, as well as to grant, reduce, revoke, or reclaim such benefits and services;
        (b) AI systems intended to be used to evaluate the creditworthiness of natural persons or establish their credit score , with the exception of AI systems used for the purpose of detecting financial fraud;
        (c) AI systems intended to evaluate and classify emergency calls by natural persons or to be used to dispatch, or to establish priority in the dispatching of emergency first response services, including by police, firefighters and medical aid, as well as of emergency healthcare patient triage systems;
        (ca) AI systems intended to be used for risk assessment and pricing in relation to natural persons in the case of life and health insurance.
      6. Law enforcement, insofar as their use is permitted under relevant Union or national law:
        (a) AI systems intended to be used by or on behalf of law enforcement authorities, or by Union institutions, agencies, offices or bodies in support of law enforcement authorities or on their behalf to assess the risk of a natural person to become a victim of criminal offences;
        (b) AI systems intended to be used by or on behalf of law enforcement authorities or by Union institutions, bodies and agencies in support of Law enforcement authorities as polygraphs and similar tools;
        (d) AI systems intended to be used by or on behalf of law enforcement authorities, or by Union institutions, agencies, offices or bodies in support of law enforcement authorities to evaluate the reliability of evidence in the course of investigation or prosecution of criminal offences;
        (e) AI systems intended to be used by law enforcement authorities or on their behalf or by Union institutions, agencies, offices or bodies in support of law enforcement authorities for assessing the risk of a natural person of offending or re-offending not solely based on profiling of natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 or to assess personality traits and characteristics or past criminal behaviour of natural persons or groups;
        (f) AI systems intended to be used by or on behalf of law enforcement authorities or by Union agencies institutions, agencies, offices or bodies in support of law enforcement authorities for profiling of natural persons as referred to in Article 3(4) of Directive (EU) 2016/680 in the course of detection, investigation or prosecution of criminal offences;
      7. Migration, asylum and border control management, insofar as their use is permitted under relevant Union or national law:
        (a) AI systems intended to be used by competent public authorities as polygraphs and similar tools;
        (b) AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies to assess a risk, including a security risk, a risk of irregular migration, or a health risk, posed by a natural person who intends to enter or has entered into the territory of a Member State;
        (d) AI systems intended to be used by or on behalf of competent public authorities or by Union agencies, offices or bodies to assist competent public authorities for the examination of applications for asylum, visa and residence permits and associated complaints with regard to the eligibility of the natural persons applying for a status, including related assessment of the reliability of evidence;
        (da) AI systems intended to be used by or on behalf of competent public authorities, including Union agencies, offices or bodies, in the context of migration, asylum and border control management, for the purpose of detecting, recognising or identifying natural persons with the exception of verification of travel documents.
      8. Administration of justice and democratic processes:
        (a) AI systems intended to be used by a judicial authority or on their behalf to assist a judicial authority in researching and interpreting facts and the law and in applying the law to a concrete set of facts or used in a similar way in alternative dispute resolution;
        (aa) AI systems intended to be used for influencing the outcome of an election or referendum or the voting behaviour of natural persons in the exercise of their vote in elections or referenda. This does not include AI systems whose output natural persons are not directly exposed to, such as tools used to organise, optimise and structure political campaigns from an administrative and logistic point of view;

        
    Article 7: Amendments to Annex III
      1. The Commission is empowered to adopt delegated acts in accordance with Article 97 to amend Annex III by adding or modifying use cases of high-risk AI systems where both of the following conditions are fulfilled:
        (a) the AI systems are intended to be used in any of the areas listed in points 1 to 8 of Annex III;
        (b) the AI systems pose a risk of harm to health and safety, or an adverse impact on fundamental rights, and that risk is equivalent to or greater than the risk of harm or of adverse impact posed by the high-risk AI systems already referred to in Annex III.
      2. When assessing for the purposes of paragraph 1 whether an AI system poses a risk of harm to the health and safety or a risk of adverse impact on fundamental rights that is equivalent to or greater than the risk of harm posed by the high-risk AI systems already referred to in Annex III, the Commission shall take into account the following criteria:
        (a) the intended purpose of the AI system;
        (b) the extent to which an AI system has been used or is likely to be used;
        (ba) the nature and amount of the data processed and used by the AI system, in particular whether special categories of personal data are processed;
        (bb) the extent to which the AI system acts autonomously and the possibility for a human to override a decision or recommendations that may lead to potential harm;
        (c) the extent to which the use of an AI system has already caused harm to health and safety, has had an adverse impact on fundamental rights or has given rise to significant concerns in relation to the likelihood of such harm or adverse impact, as demonstrated for example by reports or documented allegations submitted to national competent authorities or by other reports, as appropriate;
        (d) the potential extent of such harm or such adverse impact, in particular in terms of its intensity and its ability to affect a plurality of persons or to disproportionately affect a particular group of persons;
        (e) the extent to which potentially harmed or adversely impacted persons are dependent on the outcome produced with an AI system, in particular because for practical or legal reasons it is not reasonably possible to opt-out from that outcome;
        (f) the extent to which there is an imbalance of power, or the potentially harmed or adversely impacted persons are in a vulnerable position in relation to the user of an AI system, in particular due to status, authority, knowledge, economic or social circumstances, or age;
        (g) the extent to which the outcome produced involving an AI system is easily corrigible or reversible, taking into account the technical solutions available to correct or reverse, whereby outcomes having and adverse impact on health, safety, fundamental rights, shall not be considered as easily corrigible or reversible;
        (gb) the magnitude and likelihood of benefit of the deployment of the AI system for individuals, groups, or society at large, including possible improvements in product safety;
        (h) the extent to which existing Union legislation provides for:
          (i) effective measures of redress in relation to the risks posed by an AI system, with the exclusion of claims for damages;
          (ii) effective measures to prevent or substantially minimise those risks.
      2a. The Commission is empowered to adopt delegated acts in accordance with Article 97 to amend the list in Annex III by removing high-risk AI systems where both of the following conditions are fulfilled:
        (a) the high-risk AI system(s) concerned no longer pose any significant risks to fundamental rights, health or safety, taking into account the criteria listed in paragraph 2;
        (b) the deletion does not decrease the overall level of protection of health, safety and fundamental rights under Union law.

      Write the reasoning to be concise, fitting into one sentence of a maximum of 25 words and distilling the key info, which helps to understand why the use is Prohibiyed, High Risk or Limited or Low Risk. The format for the statement is as follows:
        if the use is classified as "Prohibited":
            "Prohibited" due to [THE REASON], which falls under the EU AI Act [RELEVANT SECTION or RULE].
        if the use is classified as "High Risk":
            "High Risk" due to [THE REASON], which falls under the EU AI Act [RELEVANT SECTION or RULE].
        or, if the classification is "Limited or Low Risk":
            "Limited or Low Risk" due to [THE REASON] [mention the EU AI Act if useful only].  
    
    Here are the details of the AI system:

        Domain: "{}",
        Purpose: "{}",
        Capability: "{}",
        AI User: "{}",
        AI Subject: "{}"

         Please return the classification in the following format:
         {{
           "Description": "The AI system intended to be used ...",
           "Classification": ["Prohibited"/"High Risk"/"Limited or Low Risk"],
           "Relevant Text from the EU AI Act": "[Quotation if applicable] - Include the Article, Annex, or Amendment or other EU AI Act section that mostly appropriately justifies the classification.",
           "Reasoning": "[CLASSIFICATION] due to [THE REASON], mention the EU AI Act if [RELEVANT SECTION or RULE]."
         }}
        
        
        Ensure to output a *CORRECTLY FORMATTED* JSON!!!
        """
    }
]



def format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject):
    S = "test {}"
    messages = deepcopy(MESSAGES)
    messages[1]['content'] = messages[1]['content'].format(domain,purpose,aiCapability,aiUser,aiSubject)
    return messages


# APPLY / RUN

In [36]:
USES[7]

{'Use': 8,
 'Domain': 'Romantic relationships and friendships',
 'Purpose': 'Planning and executing surprises',
 'Capability': 'Delivering gifts, messages',
 'AI User': 'Individuals, friends',
 'AI Subject': 'Friends, romantic partners'}

In [37]:
def parse_single_use(useElements):

    start_time = time.time()

    useI = str(useElements['Use'])
    print (f" Parsing use {useI}")

    # Variables for message placeholders
    domain = useElements['Domain']
    purpose = useElements['Purpose']
    aiCapability = useElements['Capability']
    aiUser = useElements['AI User']
    aiSubject = useElements['AI Subject']

    # Extracting "Use i" details
    use_i_details = [domain, purpose, aiCapability, aiUser, aiSubject]
    # print(use_i_details)

    # adapt the prompt for useI
    messages = format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject)
    response = chat_gpt(messages)
    print(response)
    end_time = time.time()
    print (f"This use processing took {end_time-start_time} seconds.")

    try:
        # print (type(response))
        response = ast.literal_eval(response)
        replace_key(response, "Relevant Text from the EU AI Act", "AIActText")
    except Exception as e:
        response = response.replace("```", "").replace("json", "").strip()
        response = ast.literal_eval(response)
        replace_key(response, "Relevant Text from the EU AI Act", "AIActText")

    combined_response = useElements
    for k, v in response.items():
        combined_response[k] = v

    return combined_response

In [38]:
all_uses = []
for IND in range(3):

    usee_elements = USES[IND]

    use_response= parse_single_use(usee_elements)

    all_uses.append(use_response)

 Parsing use 1
```json
{
  "Description": "The AI system intended to be used for facial recognition by security agencies and corporations to capture and analyze facial features of individuals in secured areas.",
  "Classification": "High Risk",
  "Relevant Text from the EU AI Act": "Article 6(2): High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas: Biometrics, insofar as their use is permitted under relevant Union or national law: (a) Remote biometric identification systems. This shall not include AI systems intended to be used for biometric verification whose sole purpose is to confirm that a specific natural person is the person he or she claims to be; (aa) AI systems intended to be used for biometric categorization, according to sensitive or protected attributes or characteristics based on the inference of those attributes or characteristics; (ab) AI systems intended to be used for emotion recognition.",
  "Reasoning": "High Risk due

In [40]:
all_uses

[{'Use': 1,
  'Domain': 'Biometric identification and categorization of natural persons',
  'Purpose': 'Facial recognition for security',
  'Capability': 'Capturing and analyzing facial features',
  'AI User': 'Security agencies, corporations',
  'AI Subject': 'Individuals in secured areas',
  'Description': 'The AI system intended to be used for facial recognition by security agencies and corporations to capture and analyze facial features of individuals in secured areas.',
  'Classification': 'High Risk',
  'Reasoning': 'High Risk due to involvement in biometric identification and categorization, which falls under the EU AI Act Annex III: High Risk AI Systems.',
  'AIActText': 'Article 6(2): High-risk AI systems pursuant to Article 6(2) are the AI systems listed in any of the following areas: Biometrics, insofar as their use is permitted under relevant Union or national law: (a) Remote biometric identification systems. This shall not include AI systems intended to be used for biometr

In [15]:
# results_location = "../../results/"
# FULL_RES = all_uses
# ###############################
# # save result
# with open(results_location + f"RISK_REPORT.json", "w") as json_file:
#     json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation

# Batch Processing


https://cookbook.openai.com/examples/batch_processing

In [41]:

# Creating an array of json tasks

tasks = []

for useElements in USES:

    useI = str(useElements['Use'])
    # print (f" Parsing use {useI}")

    # Variables for message placeholders
    domain = useElements['Domain']
    purpose = useElements['Purpose']
    aiCapability = useElements['Capability']
    aiUser = useElements['AI User']
    aiSubject = useElements['AI Subject']


    # adapt the prompt for useI
    messages7s = format_prompt(MESSAGES, domain,purpose,aiCapability,aiUser,aiSubject)

    
    task = {
        "custom_id": f"task-{int(useI)}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4-turbo-2024-04-09",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": messages7s,
        }
    }
    
    tasks.append(task)



# Save Batch File

In [42]:
# Creating the file

file_name = f"../../processing/batch_EUAIAct_Risks_{uses_of}.jsonl"

with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

# Uploading the file

In [43]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [44]:
print(batch_file)

FileObject(id='file-9uLgOZZmrikocaX9MgSbaDMk', bytes=5624427, created_at=1715407854, filename='batch_EUAIAct_Risks_HDI.jsonl', object='file', purpose='batch', status='processed', status_details=None)


# Creating the batch job

In [45]:
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

# Checking batch status

In [50]:
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_5vwusvFsbzvVPYbUxdn1WdOh', completion_window='24h', created_at=1715407855, endpoint='/v1/chat/completions', input_file_id='file-9uLgOZZmrikocaX9MgSbaDMk', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1715408843, error_file_id=None, errors=None, expired_at=None, expires_at=1715494255, failed_at=None, finalizing_at=1715408836, in_progress_at=1715407855, metadata=None, output_file_id='file-cvGv6ymnCaeoVZJICTvTSeBU', request_counts=BatchRequestCounts(completed=138, failed=0, total=138))


# Retrieving the results

In [51]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [52]:
result_file_name = f"../../results/batch_EUAIActRisks_results_{uses_of}.jsonl"

with open(result_file_name, 'wb') as file:
    file.write(result)

# Loading data from saved file and transforming into JSON

In [53]:
import json
import ast

In [54]:
result_file_name = f"../../results/batch_EUAIActRisks_results_{uses_of}.jsonl"

# Loading data from saved file
results = []
with open(result_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [55]:
for res in results:
    print (res)
    break

{'id': 'batch_req_hnn5IZgFneg0jjzC3Y537qZ4', 'custom_id': 'task-1', 'response': {'status_code': 200, 'request_id': '9cafb7b0c84179113e1b859ac4c11909', 'body': {'id': 'chatcmpl-9NaFYM1xWSPmZMirmU6nkfzlfwlxW', 'object': 'chat.completion', 'created': 1715407916, 'model': 'gpt-4-turbo-2024-04-09', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n  "Description": "The AI system intended to be used for facial recognition for security purposes by capturing and analyzing facial features of individuals in secured areas.",\n  "Classification": ["High Risk"],\n  "Relevant Text from the EU AI Act": "Article 6(2) and Annex III, Section 1(a) - Remote biometric identification systems.",\n  "Reasoning": "High Risk due to biometric identification in security, which falls under the EU AI Act Article 6(2) and Annex III, Section 1(a)."\n}'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 8353, 'completion_tokens': 113, 'total_tokens': 8466}, 'system_fingerp

In [56]:
FULL_RES = []
cnt_errors = []

# Reading only the first results
for res in results:
    task_id = res['custom_id']
    # Getting index from task id
    index = task_id.split('-')[-1]
    result = res['response']['body']['choices'][0]['message']['content']
    try:
        res = ast.literal_eval(result)
        # print (index)
        res["Incident ID"] = int(index)
        FULL_RES.append(res)
    except Exception as e:
        print (e)
        print (result)
        cnt_errors.append(index)
        continue
    

In [57]:
cnt_errors

[]

# SAVE

In [58]:
###############################
# save result
with open(f"../../results/FULL_{uses_of}_Risk_REPORT.json", "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation

# THE END